# Hough Transform

In [1]:
import cv2
import numpy as np

## HoughLines
`lines = cv2.HoughLines(image, rho, theta, threshold)`
- lines: outputs vector of lines in polar cooradinates
- rho: distance resolution of accumulator in pixels
- theta: angle resolution of accumulator in radians
- threshold: Only those lines are returnes that get enough votes

In [2]:
img = cv2.imread('./Pictures/sudoku.png')

# for canny edge detection convert it to grayscale img
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# lines = cv2.HoughLines(image, rho, theta, threshold)
lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

In [3]:
for line in lines:
    r, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    
    # (x0,y0) will give you the top left corner of the image
    x0 = r*a
    y0 = r*b
    
    x1 = int(x0 + 1000*b)
    y1 = int(y0 - 1000*a)
    
    x2 = int(x0 - 1000*b)
    y2 = int(y0 + 1000*a)
    
    cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 2)

In [5]:
cv2.imshow('image with HoughLines', img)
cv2.imshow('Canny edge image', edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

## HoughLinesP
`lines = cv2.HoughLinesP(image, rho, theta, threshold)`
- minLineLength: line segment shorter than this are rejected
- maxLineGap: Maximim allowed gap betweeen line segments to treat them as a single line

In [12]:
img = cv2.imread('./Pictures/road.jpg')

# for canny edge detection convert it to grayscale img
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# applying HoughLinesP
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

In [13]:
for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(img, (x1,y1), (x2,y2), (0,255,0), 2)

In [15]:
cv2.imshow('image with HoughLinesP', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Lane detection on an image

In [48]:
def roi(img, vertices):
    mask = np.zeros_like(img)
    mask_color = 255
    cv2.fillPoly(mask, vertices, mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [53]:
# read the image
img = cv2.imread('./Pictures/road.jpg')


# convert the image to grayscale mode
maskedImg_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# apply canny edge detection
canny_image = cv2.Canny(maskedImg_gray, 50, 150)


# crop the region of interest(roi)
h,w,_= img.shape
roi_vertices = [(0,h), (w/2, h/2), (w,h)]
maskedImg = roi(canny_image, np.array([roi_vertices], np.int32))


# applying HoughLinesP
lines = cv2.HoughLinesP(maskedImg, 6, np.pi/60, 160, lines=np.array([]), minLineLength=40, maxLineGap=25)

In [54]:
for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(img, (x1,y1), (x2,y2), (0,255,0), 2)

In [57]:
cv2.imshow('Original image with HoughLinesP', img)
cv2.imshow('Canny image', canny_image)
cv2.imshow('Masked image', maskedImg)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Lane detection on a video

In [3]:
def roi(img, vertices):
    mask = np.zeros_like(img)
    mask_color = 255
    cv2.fillPoly(mask, vertices, mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [54]:
def draw_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    print('-----------------------', lines)
    if lines == [None]:
        return img
    else:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(blank_image, (x1,y1), (x2,y2), (0,255,0), 3)
                #x1,y1,x2,y2 = line[0]
        img = cv2.addWeighted(img, 0.8, blank_image, 1, 0)        
        return img

In [55]:
def process(image):

    # convert the image to grayscale mode
    maskedImg_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # apply canny edge detection
    canny_image = cv2.Canny(maskedImg_gray, 100, 200)


    # crop the region of interest(roi)
    h,w,_= image.shape
    roi_vertices = [(0,h), (w/2, 3*h/4), (w,h)]
    maskedImg = roi(canny_image, np.array([roi_vertices], np.int32))


    # applying HoughLinesP
    lines = cv2.HoughLinesP(maskedImg, 6, np.pi/180, 160, minLineLength=40, maxLineGap=25)
    linesImg = draw_lines(image, np.array([lines]))
    
    return linesImg

In [56]:
# read the image
img = cv2.imread('./Pictures/road2.JPG')
img = process(img)

cv2.imshow('Original image with HoughLinesP', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

----------------------- [[[[460 378 603 494]]

  [[464 382 587 493]]]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [57]:
# read the video
cap = cv2.VideoCapture('./Videos/road.mp4')

while(cap.isOpened()):
    ret,frame = cap.read()
    frame = process(frame)
    cv2.imshow('Lane detection', frame)
    
    # exiting from the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [None]
----------------------- [[[[374 285 450 314]]]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Hough Circle Transform

In [58]:
import cv2
import numpy as np

In [59]:
img = cv2.imread('./Pictures/smarties.png')

orignal = img.copy()

# convert to grayscale and blur the image
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.medianBlur(img, 5)

# applying hough cicle transformation
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

# convert the numpy array to integers
detectedCicles = np.uint16(np.around(circles))

for (x,y,r) in detectedCicles[0,:]:
    cv2.circle(orignal, (x,y), r, (0,255,0), 3)
    cv2.circle(orignal, (x,y), 2, (0,255,255), 3)

In [60]:
cv2.imshow('orignal image', orignal)
cv2.waitKey(0)
cv2.destroyAllWindows()